# **Problema 1**
**José Antonio Torres Villegas A00835737**

### **Planteamiento del problema**

El problema lo pretendo abordar siguiendo este razonamiento: Para comprender la evolución de la criptomonedas, cuya seleccionada fue Bitcoin porque me gustaría dar con una solución robusta y que pueda tratar de generalizarse en otras acciones, estaría bien detectar alguna especie de patrón/comportamiento interesante. Para definir más cómo voy a abordar esto me voy a enfocar en el siguiente tema:

**- Tema a investigar: Detectar tendencias (Alcista, bajista o colateral)**

**- Razón de la selección: Detectar tendencias ayuda a identificar la dirección general del mercado, lo que beneficia en múltiples sentidos un análisis técnico desde evaluar el riesgo de una toma de decisión de entrar o salir del mercado hasta aumentar la precisión de modelos predictivos.**

**- Columna de enfoque: Cierre de precio (Close) del bitcoin (Son datos diarios)**

**- ¿En qué granularidad?: Se tiene una diaria, pero la idea es poder visualizarla "Mensualmente", esto se explica más en la metodología del cómo se logrará eso mismo**

### **Metodología para cumplir con la propuesta de solución**

1.- Generar 3 nuevos vectores (Columnas) para alimentar a futuros modelos: 'Diff_MA30', 'Diff_VAR30' y 'Log_Return'. Este proceso es que el que se sigue para la elaboración de los mismos y a continuación describiré el porqué de cada uno.

* a) Media y desviación estándar móvil de 30 días (Esta es la visualización "mensual"): Suaviza los datos y modela la fluctuación de los mismos usando la media y desviación estándar; este fue usado en vez de varianza para no dar mayor peso a outliers, los cuales son clave para explicar comportamientos alcistas, bajistas o colaterales (En este último, por ejemplo, suele haber una desviación estándar constante). Por otra parte la selección "mensual" es para tener un mayor rango de visión para el análiis que busque enriquecerlo y que además pueda ser de apoyo para trading a mediano o largo plazo que es lo más común.

$$\text{MA30}_t = \frac{1}{30} \sum_{i=0}^{29} \text{Close}_{t-i}$$
$$\text{VAR30}_t = \sqrt{ \frac{1}{29} \sum_{i=0}^{29} \left( \text{Close}_{t-i} - \text{MA30}_t \right)^2 }$$

* b) Diferencia relativa del precio con la media móvil mensual: Esto se hace para garantizar una mayor cercaía con una serie estacionaria en la distribución de los datos. Es un proceso muy común en series de tiempo y es parte de una buena práctica realizarla para eliminar ruido, tendencias y/o patrones cíclicos.

$$\text{Diff\_MA30}_t = \text{MA30}_t - \text{Close}_t$$
$$\text{Diff\_VAR30}_t = \text{VAR30}_t - \text{Close}_t$$

* c) Retorno logarítmico móvil: Es otro indicador sumamente usado en análisis de acciones para medir el retorno relativo y no absoluto.

$$\text{RollingLogReturn}_t = \sum_{i=0}^{29} \log \left( \frac{\text{Close}_{t-i}}{\text{Close}_{t-i-1}} \right)$$

2.- Explorar periodicidad, esencialmente con Diff_MA30 porque explica el comportamiento estructural de mejor manera tras bucar la serie estacionaria; aunque también se intentó con Diff_VAR30, ya que al haber una periodicidad aumenta también las posibilidad de obtener mejores resultados. Nota: Se asume que los datos son estacionarios, a pesar de obtener ruidos con alta frecuencia, es decir outliers en términos de análisis de datos. Esto se logra mediante:

* Implementación de encaje de Takens: Al cual se le específica encontrar los hiperparámetros óptimos para los datos del vector MA30
* Diagrama de persistencia: Haciendo uso de este encaje para corroborar si hay presencia de alguna periodicidad.

3.- Implementar un Mapper topólogico: Para observar posibles categorías (Esencialmente deberían ser 3: Los correspondientes a las tendencias esperadas, sin embargo pueden ser más por la cuestión de la cantidad de posibilidades en la interpretación de los vectores de datos para comprender la estructura particular de los precios de la acción en diversos rangos de tiempo)

4.- Observar el histórico con las categorías (Componentes conexas generadas); manteniendo el índice de los registros categorizados.

### **Conclusiones**

Se dan conclusiones a los resultados obtenidos y se da una intrepretación lo más acertada posible

## **1.- Lectura de datos y generación de vectores**

In [16]:
import pandas as pd
import numpy as np  

In [17]:
df = pd.read_csv("coin_Bitcoin.csv", usecols=["Date", "Close"], parse_dates=["Date"])
df

,Date,Close
0,2013-04-29 23:59:59,144.539993
1,2013-04-30 23:59:59,139.000000
2,2013-05-01 23:59:59,116.989998
3,2013-05-02 23:59:59,105.209999
4,2013-05-03 23:59:59,97.750000
...,...,...
2986,2021-07-02 23:59:59,33897.048590
2987,2021-07-03 23:59:59,34668.548402
2988,2021-07-04 23:59:59,35287.779766
2989,2021-07-05 23:59:59,33746.002456


In [18]:
import pandas as pd
import numpy as np

# Convirtiendo la columna de fecha a datetime
df['Date'] = pd.to_datetime(df['Date'])

# Ordenando los datos por fecha para asegurar el cálculo correcto de las ventanas móviles
df = df.sort_values('Date')

# Diferencia de 1 día del precio de cierre
df['Diff'] = df['Close'].diff()

# Calculando la media móvil y varianza móvil con ventana de 30 días
df['MA30'] = df['Close'].rolling(window=30).mean().round(2)
df['STD30'] = df['Close'].rolling(window=30).std().round(2)

# Retorno logarítmico diario
df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))

# Retorno logarítmico acumulado en ventana de 30 días
# FIX: Create column correctly instead of a row with label 'Rolling_Log_Return'
df['Rolling_Log_Return'] = df['Log_Return'].rolling(window=30).sum()

# Eliminando filas con NaN (primeros 29 días donde no hay suficientes datos para la ventana)
df_clean = df.dropna(subset=['MA30', 'STD30', 'Rolling_Log_Return'])

# Fluctuación diaria con respecto a la media móvil y varianza móvil
# FIX: Create columns correctly instead of rows
df_clean = df_clean.copy()  # Create a true copy to avoid SettingWithCopyWarning
df_clean['Diff_MA30'] = df_clean['MA30'] - df_clean['Close']
df_clean['Diff_STD30'] = df_clean['STD30'] - df_clean['Close']

# Mostrando los primeros registros con las nuevas columnas
display(df_clean)

,Date,Close,Diff,MA30,STD30,Log_Return,Rolling_Log_Return,Diff_MA30,Diff_STD30
30,2013-05-29 23:59:59,132.300003,3.300003,120.03,9.24,0.025260,-0.088484,-12.270003,-123.060003
31,2013-05-30 23:59:59,128.798996,-3.501007,119.69,8.69,-0.026819,-0.076221,-9.108996,-120.108996
32,2013-05-31 23:59:59,129.000000,0.201004,120.09,8.83,0.001559,0.097724,-8.910000,-120.170000
33,2013-06-01 23:59:59,129.300003,0.300003,120.90,8.52,0.002323,0.206177,-8.400003,-120.780003
34,2013-06-02 23:59:59,122.292000,-7.008003,121.71,7.32,-0.055724,0.223998,-0.582000,-114.972000
...,...,...,...,...,...,...,...,...,...
2986,2021-07-02 23:59:59,33897.048590,324.930937,35618.79,2402.39,0.009632,-0.103016,1721.741410,-31494.658590
2987,2021-07-03 23:59:59,34668.548402,771.499812,35467.45,2309.65,0.022505,-0.123067,798.901598,-32358.898402
2988,2021-07-04 23:59:59,35287.779766,619.231364,35413.90,2294.00,0.017704,-0.044523,126.120234,-32993.779766
2989,2021-07-05 23:59:59,33746.002456,-1541.777310,35353.70,2313.86,-0.044675,-0.052133,1607.697544,-31432.142456


## **2.- Aplicación de encaje de Takens**

Se toman en cuenta los hiperparámetros óptimos 

In [19]:
import numpy as np
import plotly.graph_objects as go
from gtda.time_series import SingleTakensEmbedding
from gtda.plotting import plot_point_cloud
import matplotlib.pyplot as plt

# Graficar los datos históricos de precio de cierre
# Convertimos a array y usamos el índice como timestamp para simplificar
close_prices = df_clean[['Close', 'Diff_MA30', 'Diff_STD30']].copy()
timestamps = np.arange(len(close_prices))

# Creando gráfico interactivo con plotly
fig = go.Figure(data=go.Scatter(x=timestamps, y=close_prices['Close'].values))
fig.update_layout(
    title="Precio de Cierre Histórico de Bitcoin",
    xaxis_title="Índice de Tiempo (Días)",
    yaxis_title="Precio de Cierre (USD)"
)
# Display the figure directly
fig.show()

# Creando gráfico interactivo con plotly
fig = go.Figure(data=go.Scatter(x=timestamps, y=close_prices['Diff_MA30'].values))
fig.update_layout(
    title="Valores de Diff_MA30",
    xaxis_title="Índice de Tiempo (Días)",
    yaxis_title="Precio de Cierre (USD)"
)
# Display the figure directly
fig.show()

# Creando gráfico interactivo con plotly
fig = go.Figure(data=go.Scatter(x=timestamps, y=close_prices['Diff_STD30'].values))
fig.update_layout(
    title="Valores de Diff_STD30",
    xaxis_title="Índice de Tiempo (Días)",
    yaxis_title="Precio de Cierre (USD)"
)
# Display the figure directly
fig.show()

# Configuración del Takens Embedding (Parametros de óptimos)
embedding_dimension = 5
embedding_time_delay = 35
stride = 5

# Crear el objeto de embedding
embedder = SingleTakensEmbedding(
    parameters_type="search",
    n_jobs=2,
    time_delay=embedding_time_delay,
    dimension=embedding_dimension,
    stride=stride,
)

# Aplicar el embedding a variables

diff_STD30_embedded = embedder.fit_transform(close_prices["Diff_STD30"])
diff_MA30_embedded = embedder.fit_transform(close_prices["Diff_MA30"])

print("Encaje de takens para Diff_STD30")
plot_point_cloud(diff_STD30_embedded)

Encaje de takens para Diff_STD30


Imagenes estáticas

<center>
    <img src="images/Precio de Cierre Histórico de Bitcoin.png" alt="Precio de Cierre Histórico de Bitcoin.png">
</center>

<center>
    <img src="images/Valores de Diff_MA30.png" alt="Valores de Diff_MA30.png">
</center>

<center>
    <img src="images/Valores de Diff_STD30.png" alt="Valores de Diff_STD30.png">
</center>

<center>
    <img src="images/Encaje de takens para Diff_STD30.png" alt="Encaje de takens para Diff_STD30.png">
</center>

In [20]:
print("Encaje de takens para Diff_MA30")
plot_point_cloud(diff_MA30_embedded)

Encaje de takens para Diff_MA30


Imagen estática

<center>
    <img src="images/Encaje de takens para Diff_MA30.png" alt="Encaje de takens para Diff_MA30.png">
</center>

Se puede observar una nubes de puntos que representa en gran medida una periodicidad

## **2.- Aplicación de diagrama de persistencia (Con Vietoris-Rips)**

In [21]:
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram

# Primero, necesitamos reformatear el embedding para el formato que espera VietorisRipsPersistence
# El formato debe ser (n_samples, n_points, n_dimensions)
diff_MA30_embedded_reshaped = diff_MA30_embedded.reshape(1, -1, diff_MA30_embedded.shape[1])
diff_STD30_embedded_reshaped = diff_STD30_embedded.reshape(1, -1, diff_STD30_embedded.shape[1])

# Definimos las dimensiones de homología que queremos calcular
# 0 - componentes conectados, 1 - ciclos, 2 - cavidades
homology_dimensions = [0, 1, 2]

# Creamos y aplicamos el cálculo de persistencia
btc_persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, 
    n_jobs=2  # Ajusta según los núcleos disponibles en tu máquina
)


print("Diagrama de Persistencia para diff_MA30")
btc_persistence.fit_transform_plot(diff_MA30_embedded_reshaped)

print("Diagrama de Persistencia para diff_STD30")
btc_persistence.fit_transform_plot(diff_STD30_embedded_reshaped)

Diagrama de Persistencia para diff_MA30


Diagrama de Persistencia para diff_STD30


array([[[0.00000000e+00, 2.85014421e-01, 0.00000000e+00],
        [0.00000000e+00, 6.25283420e-01, 0.00000000e+00],
        [0.00000000e+00, 1.92541146e+00, 0.00000000e+00],
        ...,
        [8.03576756e+00, 9.82670784e+00, 1.00000000e+00],
        [7.70408010e+00, 8.04974842e+00, 1.00000000e+00],
        [2.28534497e+03, 2.37941089e+03, 2.00000000e+00]]])

## **3.- Mapper Topológico**

In [22]:
import kmapper as km
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Preparación de datos para el mapper
# Usamos tanto la media móvil como la varianza móvil
mapper_data = df_clean[['Diff_MA30', 'Diff_STD30', 'Rolling_Log_Return']].values

# Escalar los datos
scaler = StandardScaler()
mapper_data_scaled = scaler.fit_transform(mapper_data)

# Crear y ajustar el mapper
mapper = km.KeplerMapper(verbose=1)

# Proyección - Utilizamos la suma de características 
lens = mapper.fit_transform(mapper_data_scaled, projection="l2norm")

# Definir el clustering que queremos usar (KMeans con 10 clusters)
kmeans_clusterer = KMeans(n_clusters=10, random_state=42, n_init=10)

# Crear el grafo con KMeans como algoritmo de clustering
graph = mapper.map(
    lens,
    mapper_data_scaled,
    cover=km.Cover(n_cubes=10, perc_overlap=0.4),
    clusterer=kmeans_clusterer
)

# Visualizar el grafo sin coloración personalizada
html = mapper.visualize(
    graph,
    title="Mapper Topológico - Bitcoin",
    path_html="bitcoin_mapper.html"
)

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: l2norm
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (2961, 3)

..Projecting data using: l2norm

..Scaling with: MinMaxScaler()

Mapping on data shaped (2961, 3) using lens shaped (2961, 1)

Creating 10 hypercubes.

Created 76 edges and 80 nodes in 0:00:02.620024.

Created 76 edges and 80 nodes in 0:00:02.620024.
Wrote visualization to: bitcoin_mapper.html
Wrote visualization to: bitcoin_mapper.html


<center>
    <img src="images/Mapper.png" alt="Mapper.png">
</center>

## **4.- Observar el histórico con resultados**

Aquí se tomarán en cuenta aquellos nodos interconectados y que estén en su misma "nube de puntos" como si fuera un clúster, es decir componentes conexas del mapper, para que todos aquellos registros pertenecientes a sus componentes sean los tomados en cuenta y así graficar la serie de tiempo de estos con colores distintos para visualizar mejor los resultados.

In [23]:
import kmapper as km
import networkx as nx
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Componentes del grafo
G = km.to_networkx(graph)
componentes = list(nx.connected_components(G)) 

# Índices de los datos originales
grupos_indices = []
for comp in componentes:
    indices_grupo = set()
    for nodo in comp:
        indices_grupo.update(graph['nodes'][nodo])
    grupos_indices.append(sorted(indices_grupo))  # Ordenamos para que sea más claro visualmente

# Preparar los datos base para graficar
close_prices = df_clean['Close'].values
dates = df_clean['Date'].values  # Usar fechas reales en lugar de índices

# Graficar usando Plotly
fig = go.Figure()

for i, indices in enumerate(grupos_indices):
    fig.add_trace(go.Scatter(
        x=dates[indices],
        y=close_prices[indices],
        mode='markers',
        name=f'Grupo {i+1}',
        marker=dict(size=6)
    ))

# Crear lista de fechas para los ticks anuales
min_date = df_clean['Date'].min()
max_date = df_clean['Date'].max()
# Generar fechas para el primer día de cada año en el rango de datos
year_ticks = pd.date_range(
    start=pd.Timestamp(year=min_date.year, month=1, day=1),
    end=pd.Timestamp(year=max_date.year+1, month=1, day=1),
    freq='YS'  # Inicio del año
)

fig.update_layout(
    title="Precio de Cierre de Bitcoin con Clusters del Mapper",
    xaxis=dict(
        title="Fecha",
        tickmode="array",
        tickvals=year_ticks,
        ticktext=[d.strftime('%Y') for d in year_ticks],
        tickangle=45
    ),
    yaxis_title="Precio de Cierre (USD)",
    legend_title="Clusters Topológicos"
)

# Display the figure directly instead of saving/loading as image
fig.show()

Imagen estática

<center>
    <img src="images/Precio de Cierre de Bitcoin con Clusters del Mapper.png" alt="Precio de Cierre de Bitcoin con Clusters del Mapper.png">
</center>

## **Resultados**

Fueron demasiado interesantes, satisfactorios y más de lo esperado a mí parecer, puesto que pude notar lo siguiente:

### **Periodicidades**

* **De diff_MA30:** A partir de la nube de puntos generada por el encaje de takens, se podía percibir que ya se podía contar con una y con el diagrama de persistencia se pudo confirmar eso mismo tras notar puntos, correspondiente al grupo de homología H1, ligeramente alejados al eje identidad (birth = death). 
* **De diff_STD30:** No se pudo detectar periodicidad en primera instancia tras observar la nube de puntos con el encaje de takens, sin embargo, al igual que diff_MA30, en el diagrama de persistencia se presenció un punto alejado al eje identidad corroborando que hay presencia de periodicidad.


Estas hipótesis y hallazgos hasta el momento son positivos puesto que encaminan a que este análisis tenga veracidad y fundamento. A continuación,sobre la discusión de las tendencias, se puede percibir esto mismo mediante los resultados interesantes encontrados.

### **Tendencias**

**Alcista**

El grupo 3 (Color verde claro) tuvo un rol en poder detectar comportamientos alcistas clave: Se puede observar que a medida a que se aproximan precios altos del bitcoin (Impulsos), por ejemplo alrededor del año 2018 y 2021, se tuvieron más y más presencia de estos puntos antes de esos impulsos.

**Bajista**

El grupo 2 (Color naranja) en cambio mostró poder determinar mayoritarimente zonas donde se va a tener un cambio de tendencia a la baja.

**Colateral**

Dado que los últimos datos son nuevos y no se tiene una evidencia muy larga sobre cómo se puede proseguir con la tendencia (A la baja o alza) tras romper un piso, esta resulta ser colateral: Hablando en términos de análisis técnico en donde el piso es dado por el precio que tuvo un pico en el día 17 de diciembre de 2017, cerca de 2018; poner cursor encima del punto, es decir un máximo histórico antes del gran impulso que hubo en 2021. 

Parece ser que el mapper logró entender este razonamiento muy bien y con el grupo 5 (Color naranja claro) pudo detectarlo. Hay otros grupos que sin duda alguna parecen modelar tendencia alcistas, bajistas y/o colaterales dentro de este tendencia desde un punto de vista multitemporal en análisis técnico, sin embargo estoy haciendo mención en esta discusión de resultados sobre la "imágen más grande" que modela la estructura de los precios del bitcoin.

### **Mejoras y comentarios**

Sin duda alguna hay áreas de mejoras que alcancé a detectar en los siguientes ámbitos:

* **Selección de hiperpárametros en Mapper:** El análisis se beneficiaría de una búsqueda más sistemática y exhaustiva de los hiperparámetros óptimos para el Mapper. Por ejemplo, se podría implementar una validación cruzada adaptada a series temporales para explorar diferentes combinaciones de número de cubos (n_cubes), porcentaje de traslape (perc_overlap) y funciones de proyección o lentes. Además, evaluar diferentes algoritmos de clustering como DBSCAN o Agglomerative Clustering podría mejorar la identificación de patrones no lineales en los datos.

* **Filtrado de altas frecuencias y outliers para estabilizar la estructura topológica de la serie:** Aplicar un filtrado de impulsos de alta frecuencia o valores atípicos (outliers) en la serie temporal. Estos eventos, que pueden representar ruido o errores de medición, distorsionan la estructura topológica de la señal al introducir ciclos o componentes efímeros en el diagrama de persistencia. Su eliminación mediante técnicas de suavizado o filtrado (como filtros de mediana, Hampel o wavelets) permitiría revelar patrones más estables y representativos, facilitando su identificación por el algoritmo Mapper y mejorando la calidad del análisis topológico.

* **Mayor robustez usando otros métodos tradicionales de series temporales:** Se podría integrar el enfoque topológico con métodos clásicos de análisis de series de tiempo como modelos ARIMA/SARIMA o híbridos que tienden a integrar modelos de machine learning como Random Forest o Regresión Lineal usando XGBoost, entre otros.

* **Ampliación de métricas y características:** Incluir indicadores técnicos adicionales como el RSI (Índice de Fuerza Relativa), MACD (Convergencia/Divergencia de Medias Móviles) o Bandas de Bollinger podrían enriquecer el análisis topológico. Asimismo, explorar la incorporación de datos exógenos como volumen de transacciones o eventos macroeconómicos relevantes podría mejorar la capacidad predictiva del modelo y la interpretación de las tendencias identificadas.